<h2>File paths and imports<h2>

In [ ]:
import sys 
sys.path.append("C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/ChimpRec/Code")

from chimplib.imports import *
from chimplib.recognition_utils import *

#Path to the root of the ChimpRec database ".../Chimprec Dataset/Datasets/Face Recognition/ChimpRec"
chimprec_dataset_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Datasets/Face Recognition/ChimpRec"
#Path to store the different models based on the triplet loss for ChimpRec (typically "./Code/recognition/FaceNet/Models/ChimpRec_triplet_loss")
chimprec_models_tl_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/ChimpRec/Code/recognition/FaceNet/Models/ChimpRec_triplet_loss"

#Path to the root of the CCR database".../Chimprec Dataset/Datasets/Face Recognition/CCR"
ccr_dataset_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/Chimprec Dataset/Datasets/Face Recognition/CCR"
#Path to store the different models based on the triplet loss for CCR (typically "./Code/recognition/FaceNet/Models/CCR_triplet_loss")
ccr_models_tl_path = "C:/Users/julie/OneDrive - UCL/Master_2/Mémoire/ChimpRec/Code/recognition/FaceNet/Models/CCR_triplet_loss"

<h2>Utils<h2>

In [ ]:
#Transformation to be applied to network input images
transform = transforms.Compose([
    transforms.Resize((160, 160)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
layers_to_test = [5,6,7,8,9,10,11,12,13,14,15,16,17,18]
k_to_test = [1, 3, 5]

In [ ]:
# @inputs:
# model: the model used for inference
# val_dataset_path: path to the validation dataset directory
# transform: set of transformations to apply to images (resize, normalize, ...) with torchvision.transforms
# k_vals: list of k values to evaluate (for example : [1, 3, 5])
# class_names: list of class names (individual identities) corresponding to the index labels
# all_embeddings: dictionary {class_name: [embedding vectors]} to which a new image is compared during k-NN
# has_face_labels: boolean indicating whether face labels (.txt files) are used for cropping
# @output:
# precisions: dictionary {k: float} representing the accuracy for each value of k on the validation set
def evaluate_model_on_val(model, val_dataset_path, transform, k_vals, class_names, all_embeddings, has_face_labels):
    precisions = {k: 0 for k in k_vals}
    total_img = 0

    for indiv in os.listdir(val_dataset_path):
        #The dataset structure changes depending on whether the images are already crop or if the bboxes are written to files
        indiv_path = os.path.join(val_dataset_path, indiv, "images") if has_face_labels else os.path.join(val_dataset_path, indiv)
        for img_name in os.listdir(indiv_path):
            total_img += 1
            img_path = os.path.join(indiv_path, img_name)
            image = Image.open(img_path).convert("RGB")
            
            if has_face_labels:
                #Files with annotations have the same name as images, but in txt instead of an image-specific extension (jpg, JPG, png)
                label_path = img_path.replace("images", "labels").replace(".jpg", ".txt").replace(".JPG", ".txt").replace(".png", ".txt")
                bbox = read_yolo_label(label_path, image.width, image.height)
                if bbox:
                    image = image.crop(bbox)

            for k in k_vals:
                pred = predict_face_with_tl(image, model, all_embeddings, device, transform, k, class_names)
                if pred == indiv:
                    precisions[k] += 1

    for k in precisions:
        precisions[k] /= total_img
    return precisions


In [ ]:
# @inputs:
# dataset_path: path to the root dataset directory
# model_folder: path to the directory containing the saved models
# has_face_labels: boolean indicating whether face labels (.txt files) are used for cropping
# @output:
# results: dictionary {layer: {k: precision}} containing the accuracy for each model configuration (k value and number of fine-tuned layer)
def run_experiment(dataset_path, model_folder, has_face_labels):
    train_path = os.path.join(dataset_path, "train")
    val_path = os.path.join(dataset_path, "val")
    
    #We recreate the dataset to retrieve the association between index and identity
    train_triplet_dataset = TripletLossDataset(train_path, transform=transform)
    class_names = list(train_triplet_dataset.class_to_idx.keys())

    results = {}
    for layer in layers_to_test:
        model_path = f"{model_folder}/facenet_{layer}_layers_triplet_loss.pth"
        model = InceptionResnetV1(pretrained='vggface2').eval().to(device)
        model.load_state_dict(torch.load(model_path, map_location=device))

        all_embeddings = get_all_embeddings(train_path, model, device, transform, has_face_labels)
        precisions = evaluate_model_on_val(model, val_path, transform, k_to_test, class_names, all_embeddings, has_face_labels)
        print(layer)
        print(precisions)
        results[layer] = precisions

    return results


<h2>Calculating model scores<h2>

<h3>ChimpRec<h3>

In [ ]:
run_experiment(chimprec_dataset_path, chimprec_models_tl_path, True)

<h3>CCR<h3>

In [ ]:
run_experiment(ccr_dataset_path, ccr_models_tl_path, False)

<h2>Graphs<h2>

In [ ]:
# @inputs:
# results: dictionary {layer: {k: precision}} generated by run_experiment
# dataset_name: name of the dataset on wich the results have been calculated
# file_save_path: path where the accuracy plot image should be saved
# @output:
# None (saves and displays a plot of accuracy for different hyperparameter configurations)
def show_graph(results, dataset_name, file_save_path):
    layers = list(results.keys())

    plt.figure(figsize=(10, 6))

    for k in k_to_test:
        values = [results[layer][k] for layer in layers]
        plt.plot(layers, values, marker='o', label=f'k = {k}')

    plt.xlabel('#Fine-tuned layers')
    plt.ylabel('Accuracy')
    plt.title(f'Evolution of the accuracy on the {dataset_name} validation dataset according to the value of the hyperparameters')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(file_save_path)
    plt.show()

<h3>ChimpRec<h3>

In [ ]:
#Values obtained when calculating accuracy for all configurations with the ChimpRec model (to avoid having to rerun all the code each time)
chimprec_results = {5: {1: 0.7533039647577092, 3: 0.7180616740088106, 5: 0.7136563876651982}, 6: {1: 0.8810572687224669, 3: 0.8854625550660793, 5: 0.8854625550660793}, 7: {1: 0.8193832599118943, 3: 0.7929515418502202, 5: 0.8193832599118943}, 8: {1: 0.8546255506607929, 3: 0.8502202643171806, 5: 0.8414096916299559}, 9: {1: 0.8193832599118943, 3: 0.8193832599118943, 5: 0.8237885462555066}, 10: {1: 0.7048458149779736, 3: 0.7092511013215859, 5: 0.7136563876651982}, 11: {1: 0.8061674008810573, 3: 0.8105726872246696, 5: 0.801762114537445}, 12: {1: 0.8986784140969163, 3: 0.8722466960352423, 5: 0.8766519823788547}, 13: {1: 0.8810572687224669, 3: 0.8898678414096917, 5: 0.8898678414096917}, 14: {1: 0.8061674008810573, 3: 0.8105726872246696, 5: 0.8193832599118943}, 15: {1: 0.8634361233480177, 3: 0.8502202643171806, 5: 0.8590308370044053}, 16: {1: 0.9118942731277533, 3: 0.9030837004405287, 5: 0.8898678414096917}, 17: {1: 0.7577092511013216, 3: 0.7444933920704846, 5: 0.7577092511013216}, 18: {1: 0.8414096916299559, 3: 0.8502202643171806, 5: 0.8502202643171806}}

In [ ]:
show_graph(chimprec_results, "ChimpRec", "C:/Users/julie/Documents/Unif/Test_mémoire/fine_tuning_ChimpRec_triplet_loss.svg")

<h3>CCR<h3>

In [ ]:
#Values obtained when calculating accuracy for all configurations with the CCR model (to avoid having to rerun all the code each time)
ccr_results = {5: {1: 0.296, 3: 0.3028, 5: 0.3116}, 6: {1: 0.4876, 3: 0.5016, 5: 0.5088}, 7: {1: 0.4252, 3: 0.4424, 5: 0.4452}, 8: {1: 0.6112, 3: 0.6108, 5: 0.6152}, 9: {1: 0.4064, 3: 0.4104, 5: 0.4224}, 10: {1: 0.3344, 3: 0.3484, 5: 0.3548}, 11: {1: 0.5004, 3: 0.5068, 5: 0.5016}, 12: {1: 0.5428, 3: 0.542, 5: 0.5416}, 13: {1: 0.636, 3: 0.632, 5: 0.6332}, 14: {1: 0.5276, 3: 0.5352, 5: 0.5352}, 15: {1: 0.52, 3: 0.5236, 5: 0.5192}, 16: {1: 0.5096, 3: 0.5176, 5: 0.5276}, 17: {1: 0.4068, 3: 0.4188, 5: 0.42}, 18: {1: 0.4576, 3: 0.4596, 5: 0.4604}}

In [ ]:
show_graph(ccr_results, "CCR", "C:/Users/julie/Documents/Unif/Test_mémoire/fine_tuning_CCR_triplet_loss.svg")